<a href="https://colab.research.google.com/github/tintamaria95/CocoriGo/blob/resnet_bl/Projet_GO_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DL Project

##Golois import

In [1]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

--2022-11-10 12:07:37--  https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138784129 (132M) [application/zip]
Saving to: ‘project2022.zip’

project2022.zip     100%[===================>] 132.35M  18.3MB/s    in 8.1s    

2022-11-10 12:07:46 (16.4 MB/s) - ‘project2022.zip’ saved [138784129/138784129]

Archive:  project2022.zip
  inflating: Board.h                 
  inflating: Game.h                  
  inflating: Rzone.h                 
  inflating: compile.sh              
  inflating: compileMAC.sh           
  inflating: games.data              
  inflating: golois.cpp              
  inflating: golois.cpython-310-x86_64-linux-gnu.so  
  inflating: golois.cpython-37m-x86_64-linux-gnu.so  
  inflating: golois.cpython-38-x86_64-linux-gnu.so  
  inflat

In [2]:
!python golois.py

getValidation
r.shape = (10000, 19, 19, 31)
nbExamples = 10000
tcmalloc: large alloc 2400002048 bytes == 0x5744e000 @  0x7f1ac7c42887 0x7f1a4d7b70d9 0x7f1a4d7bc85f 0x7f1a4d7d106f 0x58f6e4 0x5105e2 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7f1ac783dc87 0x5b636a
nbPositionsSGF = 29425326
nbPositionsSGF = 29425326
generating validation.data
2022-11-10 12:08:34.866373: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 board (InputLayer)             [(None, 19, 19, 31)  0           []                               
                                ]                                                                 
                                                                

##Neural Network

In [10]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers
import gc

import golois

In [11]:
planes = 31
moves = 361
N = 10000
epochs = 10
batch = 128
filters = 32

In [12]:
input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

getValidation


In [13]:
input = keras.Input(shape=(19, 19, planes), name='board')

x_1 = layers.Conv2D(filters, 5, activation='relu', padding='same')(input)
x_2 = layers.Conv2D(filters, 5, activation='relu', padding='same')(x_1)
x_3 = layers.Conv2D(filters, 5, activation='relu', padding='same')(x_2)

y_1 = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
y_2 = layers.Conv2D(filters, 1, activation='relu', padding='same')(y_1)
y_3 = layers.Conv2D(filters, 1, activation='relu', padding='same')(y_2)

inter_output = tf.keras.layers.Add()([x_3, y_3])

z = tf.keras.layers.ReLU()(inter_output)

policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(z)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)
value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(z)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 1.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 board (InputLayer)             [(None, 19, 19, 31)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_6 (Conv2D)              (None, 19, 19, 32)   24832       ['board[0][0]']                  
                                                                                                  
 conv2d_9 (Conv2D)              (None, 19, 19, 32)   1024        ['board[0][0]']                  
                                                                                                  
 conv2d_7 (Conv2D)              (None, 19, 19, 32)   25632       ['conv2d_6[0][0]']         

In [ ]:
for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups, i * N)
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, batch_size=batch)
    if (i % 5 == 0):
        gc.collect ()
    if (i % 20 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)
        print ("val =", val)
        model.save ('test.h5')

epoch 1
79/79 [==============================] - 70s 872ms/step - loss: 6.6032 - policy_loss: 5.8968 - value_loss: 0.6971 - policy_categorical_accuracy: 0.0022 - value_mse: 0.1234
epoch 2
79/79 [==============================] - 67s 849ms/step - loss: 6.5939 - policy_loss: 5.8889 - value_loss: 0.6956 - policy_categorical_accuracy: 0.0030 - value_mse: 0.1206
epoch 3
79/79 [==============================] - 64s 814ms/step - loss: 6.5668 - policy_loss: 5.8632 - value_loss: 0.6941 - policy_categorical_accuracy: 0.0063 - value_mse: 0.1210
epoch 4
79/79 [==============================] - 65s 819ms/step - loss: 6.4559 - policy_loss: 5.7511 - value_loss: 0.6953 - policy_categorical_accuracy: 0.0114 - value_mse: 0.1218
epoch 5
60/79 [=====================>........] - ETA: 16s - loss: 6.3294 - policy_loss: 5.6251 - value_loss: 0.6949 - policy_categorical_accuracy: 0.0099 - value_mse: 0.1202